In [24]:
import pandas as pd
import os
from google import genai
from docx import Document

In [25]:
#Cambiar directorio de trabajo
os.chdir(r"H:/Mi unidad\pgd\informacion_corto_plazo")

In [26]:
##Leer matriz de consolidación
matriz=pd.read_excel("matriz_consolidacion.xlsx", usecols=range(8))
#Leer credenciales de google

with open('C:/cred_google_gemini.txt', 'r') as file:
    llave = file.read()

matriz

,linea,eje,tema,documento,variable,texto,num_pag,url
0,Plena de derechos de infancia y juventudes,"Educación, cultura, disfrute, salud y cuidados...",Educación,Programa provisional de gobierno del Gobierno ...,Diagnóstico,Datos de la Secretaría de Educación Pública in...,166,https://www.congresocdmx.gob.mx/media/document...
1,Plena de derechos de infancia y juventudes,"Educación, cultura, disfrute, salud y cuidados...",Educación,Programa provisional de gobierno del Gobierno ...,Diagnóstico,Por otra parte según datos de la Secretaría d...,198,https://www.congresocdmx.gob.mx/media/document...
2,Plena de derechos de infancia y juventudes,"Educación, cultura, disfrute, salud y cuidados...",Educación,Programa provisional de gobierno del Gobierno ...,Objetivo,Garantizar que todas las personas en la Ciudad...,169,https://www.congresocdmx.gob.mx/media/document...
3,Plena de derechos de infancia y juventudes,"Educación, cultura, disfrute, salud y cuidados...",Educación,Programa provisional de gobierno del Gobierno ...,Objetivo,Garantizar el ejercicio pleno de los derechos ...,199,https://www.congresocdmx.gob.mx/media/document...
4,Plena de derechos de infancia y juventudes,"Educación, cultura, disfrute, salud y cuidados...",Educación,Programa provisional de gobierno del Gobierno ...,Estrategia corto plazo,Rehabilitar la infraestructura escolar e impul...,170,https://www.congresocdmx.gob.mx/media/document...
...,...,...,...,...,...,...,...,...
64,Plena de derechos de infancia y juventudes,"Educación, cultura, disfrute, salud y cuidados...",Educación,Programa de gobierno de Clara Brugada 2024-2030,Estrategia corto plazo,Equiparemos a las escuelas con instrumentos mu...,12,Clara-Brugada-Programa-de-Gobierno-2024-2030.pdf
65,Plena de derechos de infancia y juventudes,"Trabajo y vivienda, para juventudes",Trabajo,Programa de gobierno de Clara Brugada 2024-2030,Diagnóstico,na,na,Clara-Brugada-Programa-de-Gobierno-2024-2030.pdf
66,Plena de derechos de infancia y juventudes,"Trabajo y vivienda, para juventudes",Trabajo,Programa de gobierno de Clara Brugada 2024-2030,Objetivo,na,na,Clara-Brugada-Programa-de-Gobierno-2024-2030.pdf
67,Plena de derechos de infancia y juventudes,"Trabajo y vivienda, para juventudes",Trabajo,Programa de gobierno de Clara Brugada 2024-2030,Estrategia corto plazo,Crearemos un gran programa integral para jóven...,15,Clara-Brugada-Programa-de-Gobierno-2024-2030.pdf


In [27]:
# 2) Función que arma el “documento” por grupo
def documento_por_grupo(g, salto="\n\n"):
    diag = salto.join(
        f"Diagnóstico: {t.strip()}"
        for t in g.loc[g.variable.str.contains("Diagnóstico", na=False), "texto"]
    )
    obj  = salto.join(
        f"Objetivo: {t.strip()}"
        for t in g.loc[g.variable.str.contains("Objetivo", na=False), "texto"]
    )
    est  = salto.join(
        f"Estrategia: {t.strip()}"
        for t in g.loc[g.variable.str.contains("Estrategia", na=False), "texto"]
    )
    return diag, obj, est

In [28]:
grupos = (
    matriz.groupby(["linea"], dropna=False)
          .apply(documento_por_grupo)
)

print(f"Se generarán {len(grupos)} documentos Word")
grupos

Se generarán 1 documentos Word


C:\Users\claud\AppData\Local\Temp\ipykernel_7644\791132919.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(documento_por_grupo)


linea
Plena de derechos de infancia y juventudes    (Diagnóstico: Datos de la Secretaría de Educac...
dtype: object

In [29]:
# 4. Crear carpeta de salida si no existe
if not os.path.exists("resumenes_docx"):
    os.makedirs("resumenes_docx")

In [30]:
client=genai.Client(api_key=llave)

In [33]:
for clave, (diag, obj, est) in grupos.items():
    linea = clave

    # --- Prompt con prefijos ya incluidos -----------------------------------
    prompt = (
        "Redacta un documento extenso en donde se utilice la siguiente información:\n\n"
        f"{diag}\n\n{obj}\n\n{est}"
        "No redactes en primera persona, ni uses el pronombre 'yo' No copies íntegramente las redacciones, modifícalas, sobre todo en el diagnóstico, pero recuerda que debe tener coherencia.\n"
        "La estructura del documento debe ser la siguiente:\n"
        "Diagnóstico: [texto del diagnóstico] El diagnóstico no debe ser de más de dos cuartillas\n" \
        "Objetivos de corto plazo: [texto del objetivo]. No establezcas temporalidad en el título y haz una redacción que no sea una copia íntegra pero sin perder el sentido\n" \
        "Estrategias de corto plazo: [texto de la estrategia]. Redáctalas de una forma en que no sea una copia íntegra\n\n"
    )

    resp = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[prompt]
    )
    resumen = resp.candidates[0].content.parts[0].text.strip()


    # --- Crear DOCX ----------------------------------------------------------
    doc = Document()
    doc.add_heading(f"{linea}", level=1)
    #doc.add_paragraph(f"Eje: {eje}")
    #doc.add_paragraph(f"Documento fuente: {documento}")
    doc.add_heading("Documento", level=2)
    doc.add_paragraph(resumen)

    # Anexo con texto completo
    doc.add_page_break()
    doc.add_heading("Anexo: texto íntegro", level=2)
    if diag: doc.add_paragraph(diag)
    if obj:  doc.add_paragraph("\n" + obj)
    if est:  doc.add_paragraph("\n" + est)

    # Guardar
    nombre = f"{linea[:30]}.docx".replace(" ", "_")
    #nombre = f"{linea[:30]}_{tema[:20]}.docx".replace(" ", "_")
    ruta = os.path.join("resumenes_docx", nombre)
    doc.save(ruta)
    print("✓ Guardado:", ruta)

✓ Guardado: resumenes_docx\Plena_de_derechos_de_infancia_.docx
